# English to Twi Translation Model

This notebook demonstrates the fine-tuning of a pretrained Hugging Face model to translate English sentences into Twi, a native language of Ghana. By leveraging a state-of-the-art transformer-based model, I aim to understand and recongnize the value of machine translation - a task that presents unique linguistic challenges due to the differences in grammatical structure and vocabulary between the two languages.



In [1]:
# import libraries

import pandas as pd
from sklearn.model_selection import train_test_split


## Data Sourcing

The data used in this task is sourced from Zenodo and consists of a dataset containing over 25,000 English sentences and their corresponding translations in the Twi language. 

In [2]:
# Get data

!wget -O data.csv https://zenodo.org/records/4432117/files/verified_data.csv?download=1

--2024-10-05 12:12:10--  https://zenodo.org/records/4432117/files/verified_data.csv?download=1
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.103.159, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1795088 (1.7M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.71M  2.31MB/s    in 0.7s    

2024-10-05 12:12:12 (2.31 MB/s) - ‘data.csv’ saved [1795088/1795088]



In [3]:
# load csv data

df = pd.read_csv('data/data.csv')
df = df.rename(columns={"English": "input", "Akuapem Twi": "target"})

df

,input,target
0,What she lacks in charisma she makes up for wi...,Nea onni ho adwempa no de adwumaden na ɛba.
1,There was nothing I could do about it.,Na biribiara nni hɛ a metumi ayɔ
2,Kwaku saw John and Abena holding hands.,Kwaku hui se John ne Abena kurakura wɛn nsa.
3,Can you stay till 2:30?,So wubetumi atena ha akosi nnɛnmienu npaamu ad...
4,You haven't got much time.,Wonni mmre
...,...,...
25415,I'm not a killer,menyɛ owudini
25416,what are you searching for?,dɛn na worehwehwɛ?
25417,kwabena went out,Kwabena fii adi.
25418,kwabena stepped outside,Kwabena sii aduo


### Train test validation splitting using sklearn

In [4]:
# Split the data into train, validation, and test sets (80%, 10%, 10%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}, Test size: {len(test_df)}")

Train size: 20336, Validation size: 2542, Test size: 2542


## Using a Pre-trained Model - MarianMTModel

I am using a pretrained model because it allows me to build on the extensive language knowledge the model has already acquired from large-scale multilingual datasets. Pretrained models like those from Hugging Face's library have learned general language patterns, structures, and representations from diverse text corpora. By starting with a model that already understands the fundamentals of language translation, I can focus on fine-tuning it for the specific task of translating English to Twi, which significantly reduces the data requirements and training time compared to training a model from scratch.

Why I Chose MarianMTModel

I chose MarianMTModel because it is a specialized transformer-based model designed specifically for machine translation tasks. Here’s why it is ideal for this project:

- Multilingual Translation: MarianMTModel supports a wide range of language pairs, including low-resource languages like Twi. This makes it a perfect fit for translating to a language with limited training data.

- Pretrained for Translation: MarianMTModel is already optimized for translation tasks, unlike general models such as BERT or GPT. This makes it more effective for sentence-level translations, capturing the linguistic differences between English and Twi.

- Efficiency: MarianMTModel is lightweight and efficient, making it suitable for fine-tuning even on a smaller subset of the dataset. Given my choice to work with 5,000 data points, its efficiency is crucial for handling the task without excessive computational overhead.

Using MarianMTModel enables me to take advantage of its pre-existing translation capabilities while fine-tuning it for the specific needs of English to Twi translation.

In [5]:
from transformers import MarianMTModel, MarianTokenizer

# Load a pre-trained MarianMT model and tokenizer
model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/Users/kingsleybudu/dev/projects/twi_translation/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kingsleybudu/dev/projects/twi_translation/nlp/lib/python3.9/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/Users/kingsleybudu/dev/projects/twi_translation/nlp/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
def tokenize_sentences(df, tokenizer, max_len=128):
    '''
    Tokenize the input and target columns

    Args:
        df -> pandas dataframe containing the input and the target
        tokenizer -> tokenizer instance used to tokenize the corpus
        max_len -> maximum number of tokens

    Returns:
        input encodings
        target encodings
    '''
    input_encodings = tokenizer(list(df['input']), padding=True, truncation=True, max_length=max_len, return_tensors='pt')
    target_encodings = tokenizer(list(df['target']), padding=True, truncation=True, max_length=max_len, return_tensors='pt')

    return input_encodings, target_encodings

train_input_encodings, train_target_encodings = tokenize_sentences(train_df, tokenizer)
val_input_encodings, val_target_encodings = tokenize_sentences(val_df, tokenizer)

In [7]:
import torch
from torch.utils.data import Dataset

class TranslationDataset(Dataset):
    """
    A custom Dataset class for handling translation tasks.

    This class inherits from torch.utils.data.Dataset and is used to prepare
    tokenized input and target data for a translation model
    """

    def __init__(self, input_encodings, target_encodings):
        '''
        class constructor

        Args:
            input_encodings (dict): A dictionary containing the tokenized input (source language) data.
            target_encodings (dict): A dictionary containing the tokenized target (translated language) data.
        '''
        self.input_encodings = input_encodings
        self.target_encodings = target_encodings

    def __len__(self):
        """ Returns the number of samples in the dataset (based on input encodings length) """
        return len(self.input_encodings['input_ids'])

    def __getitem__(self, idx):
        """ Retrieves a single sample of input and target encodings as tensors, for use by the model. """
        item = {key: torch.tensor(val[idx]) for key, val in self.input_encodings.items()}
        item['labels'] = torch.tensor(self.target_encodings['input_ids'][idx])
        return item

train_dataset = TranslationDataset(train_input_encodings, train_target_encodings)
val_dataset = TranslationDataset(val_input_encodings, val_target_encodings)

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

/Users/kingsleybudu/dev/projects/twi_translation/nlp/lib/python3.9/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/5084 [00:00<?, ?it/s]/var/folders/lv/jcj3v7t92nv_xc98786nj4wm0000gn/T/ipykernel_64428/3969122050.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.input_encodings.items()}
/var/folders/lv/jcj3v7t92nv_xc98786nj4wm0000gn/T/ipykernel_64428/3969122050.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.target_encodi

{'loss': 2.0769, 'grad_norm': 2.7659943103790283, 'learning_rate': 4.9901652242328875e-05, 'epoch': 0.0}


  0%|          | 20/5084 [01:52<18:40:35, 13.28s/it]

{'loss': 1.4154, 'grad_norm': 2.5821735858917236, 'learning_rate': 4.9803304484657754e-05, 'epoch': 0.01}


  1%|          | 30/5084 [02:23<4:22:02,  3.11s/it] 

...

100%|█████████▉| 5080/5084 [3:59:12<00:09,  2.37s/it]

{'loss': 0.2386, 'grad_norm': 2.2725095748901367, 'learning_rate': 3.933910306845004e-08, 'epoch': 2.0}


100%|██████████| 5084/5084 [3:59:21<00:00,  2.36s/it]/var/folders/lv/jcj3v7t92nv_xc98786nj4wm0000gn/T/ipykernel_64428/3969122050.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.input_encodings.items()}
/var/folders/lv/jcj3v7t92nv_xc98786nj4wm0000gn/T/ipykernel_64428/3969122050.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.target_encodings['input_ids'][idx])
                                                     
100%|██████████| 5084/5084 [4:02:34<00:00,  2.86s/it]

{'eval_loss': 0.27893632650375366, 'eval_runtime': 188.5866, 'eval_samples_per_second': 13.479, 'eval_steps_per_second': 1.686, 'epoch': 2.0}
{'train_runtime': 14554.1039, 'train_samples_per_second': 2.795, 'train_steps_per_second': 0.349, 'train_loss': 0.37589631002780305, 'epoch': 2.0}


TrainOutput(global_step=5084, training_loss=0.37589631002780305, metrics={'train_runtime': 14554.1039, 'train_samples_per_second': 2.795, 'train_steps_per_second': 0.349, 'total_flos': 463161834602496.0, 'train_loss': 0.37589631002780305, 'epoch': 2.0})

In [9]:
test_input_encodings, test_target_encodings = tokenize_sentences(test_df, tokenizer)
test_dataset = TranslationDataset(test_input_encodings, test_target_encodings)

# Evaluate the model
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print(f"Evaluation Results: {eval_results}")

/var/folders/lv/jcj3v7t92nv_xc98786nj4wm0000gn/T/ipykernel_64428/3969122050.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.input_encodings.items()}
/var/folders/lv/jcj3v7t92nv_xc98786nj4wm0000gn/T/ipykernel_64428/3969122050.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.target_encodings['input_ids'][idx])
100%|██████████| 318/318 [03:31<00:00,  1.50it/s]

Evaluation Results: {'eval_loss': 0.3047913610935211, 'eval_runtime': 212.4013, 'eval_samples_per_second': 11.968, 'eval_steps_per_second': 1.497, 'epoch': 2.0}


## Text Translation

The following code defines the translate function, which takes an English sentence as input and returns its Twi translation. The function tokenizes the input text, uses the fine-tuned MarianMTModel to generate the translation, and then decodes the output back into readable Twi text. This function is essential for performing the actual translation task using the model and tokenizer.

In [10]:
def translate(text, trainer, tokenizer):
    '''
    Function that translates a given text to Twi

    Args:
        text -> the text to be translated
        trainer -> trainer instance that contains the model
        tokenizer -> tokenizer instance to tokenize the text

    Returns:
        Transalted text
    '''
    # Extract the model from the trainer
    model = trainer.model

    input_encodings = tokenizer(text, return_tensors='pt', padding=True)

    # Generate translation
    translated_tokens = model.generate(**input_encodings)

    # Decode the output
    translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]

    return translated_text

# Test
text_to_translate = ["Kwaku saw John and Abena holding hands."]
translated_text = translate(text_to_translate, trainer, tokenizer)
print(f"Translation: {translated_text}")

Translation: ['Kwaku huu John ne Abena wɔn nsam.']


In [11]:
trainer.save_model('model/')
tokenizer.save_pretrained('model/')

('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/vocab.json',
 'model/source.spm',
 'model/target.spm',
 'model/added_tokens.json')

## Load and Translate

In [13]:
from transformers import MarianMTModel, MarianTokenizer

# Load the pretrained model and tokenizer
model_name = "model/"
tokenizer_name = "model/"

model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(tokenizer_name)

def translate(text, model, tokenizer):
    '''
    Function that translates a given text to Twi

    Args:
        text -> the text to be translated
        trainer -> trainer instance that contains the model
        tokenizer -> tokenizer instance to tokenize the text

    Returns:
        Transalted text
    '''
    input_encodings = tokenizer(text, return_tensors='pt', padding=True)

    # Generate translation
    translated_tokens = model.generate(**input_encodings)

    # Decode the output
    translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]

    return translated_text

# Example translation
text_to_translate = ["I am quite hungry"]
translated_text = translate(text_to_translate, model, tokenizer)
print(f"Translation: {translated_text}")


/Users/kingsleybudu/dev/projects/twi_translation/nlp/lib/python3.9/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Translation: ['Ɔkɔm de me paa.']
